In [1]:
# 2020-01-01 to 2020-05-02
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
print(datetime.datetime.now())


date_start=datetime.date(2020,1,1)
date_end=datetime.date(2020,5,2)
date_start,date_end

2020-05-27 11:18:10.982909


(datetime.date(2020, 1, 1), datetime.date(2020, 5, 2))

In [5]:
list_dcm_impr_logs=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/DCM_raw_logs_BL/impressions/*")
list_dcm_impr_logs=[x for x in list_dcm_impr_logs if "." in x]
print(len(list_dcm_impr_logs))
print(set([x.split(".")[1] for x in list_dcm_impr_logs]))
list_dcm_impr_logs.sort()

df_impr_dcm_logs=pd.DataFrame({"file_path":list_dcm_impr_logs})
df_impr_dcm_logs['utc_date']=df_impr_dcm_logs['file_path'].apply(lambda x: x.split("impression_utc_")[1][:8])
df_impr_dcm_logs=df_impr_dcm_logs[df_impr_dcm_logs['utc_date']>=str(date_start).replace("-","")]
df_impr_dcm_logs=df_impr_dcm_logs[df_impr_dcm_logs['utc_date']<=str(date_end).replace("-","")]

print(df_impr_dcm_logs.shape,df_impr_dcm_logs['utc_date'].min(),df_impr_dcm_logs['utc_date'].max(),df_impr_dcm_logs['utc_date'].nunique())

711
{'tsv'}
(123, 2) 20200101 20200502 123


In [7]:
list_lr_impr_returned=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/*")
print(len(list_lr_impr_returned))
print(set([x.split(".")[1] for x in list_lr_impr_returned]))
list_lr_impr_returned.sort()

df_impr_lr_returned=pd.DataFrame({"file_path":list_lr_impr_returned})
df_impr_lr_returned['utc_date']=df_impr_lr_returned['file_path'].apply(lambda x: x.split("account7252_impression_")[1][:8])
df_impr_lr_returned=df_impr_lr_returned[df_impr_lr_returned['utc_date']>=str(date_start).replace("-","")]
df_impr_lr_returned=df_impr_lr_returned[df_impr_lr_returned['utc_date']<=str(date_end).replace("-","")]
print(df_impr_lr_returned.shape,df_impr_lr_returned['utc_date'].min(),df_impr_lr_returned['utc_date'].max(),df_impr_lr_returned['utc_date'].nunique())



10023
{'tsv'}
(115, 2) 20200101 20200502 115


In [9]:
list_common_dates=set(df_impr_lr_returned['utc_date'].unique().tolist()).intersection(set(df_impr_dcm_logs['utc_date'].unique().tolist()))
list_common_dates=list(list_common_dates)
list_common_dates.sort()
print(len(list_common_dates))

115


In [10]:
writer_mapping="/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/"
try:
    os.stat(writer_mapping)
except:
    os.mkdir(writer_mapping)

In [22]:
df_uid_idl_mapping=pd.DataFrame()
error_file_list_lr_returned=[]
error_file_list_dcm_logs=[]
i_counter=0
for date in list_common_dates:
    i_counter+=1
    print(i_counter,date,datetime.datetime.now())
    
    df_lr_returned=pd.DataFrame()
    df_dcm_logs=pd.DataFrame()
    
    file_list_lr=df_impr_lr_returned[df_impr_lr_returned['utc_date']==date]['file_path'].tolist()
    file_list_dcm=df_impr_dcm_logs[df_impr_dcm_logs['utc_date']==date]['file_path'].tolist()
    
    for file in file_list_lr:
        try:
            df=pd.read_csv(file,dtype=str,sep="\t")
            df=df[df['Advertiser ID']=="8095847"]
            df_lr_returned=df_lr_returned.append(df)
        except:
            try:
                print("eof error, read only 10**4 rows")
                df=pd.read_csv(file,dtype=str,sep="\t",nrows=10**4)
                df=df[df['Advertiser ID']=="8095847"]
                df_lr_returned=df_lr_returned.append(df)
            except:
                error_file_list_lr_returned.append(file)
                print("error_file_list_lr_returned: ",file)
    
    for file in file_list_dcm:
        try:
            df=pd.read_csv(file,dtype=str,sep="\t")
            df=df[df['Advertiser ID']=="8095847"]
            df_dcm_logs=df_dcm_logs.append(df)
        except:
            error_file_list_dcm_logs.append(file)
            print("error_file_list_dcm_logs: ",file)
            
        
    merge_col_list=df_dcm_logs.columns.tolist()
    for col in ['User ID', 'Partner1 ID',"Partner2 ID"]:
        try:
            merge_col_list.remove(col)
        except:
            pass
    df_lr_returned=df_lr_returned[['Customer_Link']+merge_col_list]
    df_lr_returned=df_lr_returned.drop_duplicates()

    df_lr_returned=df_lr_returned[df_lr_returned['Customer_Link']!="UNMATCHED"]
    
    df_dcm_logs=df_dcm_logs.drop_duplicates()
    
    df_mapping=pd.merge(df_dcm_logs,df_lr_returned,on=merge_col_list,how="inner")
    df_mapping=df_mapping[['Customer_Link','User ID']]
    df_mapping['file_date']=date
    
    df_uid_idl_mapping=df_uid_idl_mapping.append(df_mapping)
    df_uid_idl_mapping=df_uid_idl_mapping.drop_duplicates(['Customer_Link','User ID'])
    

date_min=df_uid_idl_mapping['file_date'].min()
date_max=df_uid_idl_mapping['file_date'].max()

df_uid_idl_mapping.to_csv(writer_mapping+"BL_GoogleID_IDL_mapping_based_on_imrp_%s_%s_JL_%s.csv" % (str(date_min),str(date_max),str(datetime.datetime.now().date())),index=False)
gc.collect()

1 20200101 2020-05-27 12:47:46.801047
2 20200102 2020-05-27 12:50:31.237499
3 20200103 2020-05-27 12:50:49.983072
4 20200104 2020-05-27 12:51:00.613941
5 20200105 2020-05-27 12:51:08.776621
6 20200106 2020-05-27 12:51:14.805848
7 20200107 2020-05-27 12:51:20.795599
8 20200108 2020-05-27 12:51:27.932580
eof error, read only 10**4 rows
9 20200109 2020-05-27 12:51:35.435896
10 20200110 2020-05-27 12:51:50.102090
11 20200111 2020-05-27 12:52:03.353488
12 20200112 2020-05-27 12:52:15.582120
13 20200113 2020-05-27 12:52:18.161264
14 20200114 2020-05-27 12:52:19.214978
15 20200115 2020-05-27 12:52:20.523161
16 20200116 2020-05-27 12:52:38.245421
17 20200117 2020-05-27 12:53:16.199873
18 20200118 2020-05-27 12:53:48.679372
19 20200119 2020-05-27 12:54:21.195014
20 20200120 2020-05-27 12:55:00.138014
21 20200121 2020-05-27 12:56:35.311797
22 20200122 2020-05-27 12:56:37.927709
23 20200130 2020-05-27 12:56:39.259817
24 20200131 2020-05-27 12:56:40.559096
25 20200202 2020-05-27 12:56:41.908908
26

0

In [23]:
print("len(error_file_list_lr_returned)",len(error_file_list_lr_returned))
print("len(error_file_list_dcm_logs)",len(error_file_list_dcm_logs))
df_error_file_list_lr_returned=pd.DataFrame({"file_path":error_file_list_lr_returned})
df_error_file_list_lr_returned.to_csv(writer_mapping+"df_error_file_list_lr_returned.csv",index=False)

len(error_file_list_lr_returned) 0
len(error_file_list_dcm_logs) 0


In [24]:
# looks all right since multiple files exising, and the errro is mostly about the header name
df_impr_lr_returned[df_impr_lr_returned['utc_date']=="20190321"].shape

(0, 2)

In [25]:
writer_mapping

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/'

In [26]:
df_uid_idl_mapping_2020_impr=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/BL_GoogleID_IDL_mapping_based_on_imrp_20200101_20200502_JL_2020-05-27.csv",
                               dtype=str)

In [27]:
df_uid_idl_mapping_2020_impr.shape

(6778865, 3)

In [28]:
df_uid_idl_mapping_2020_impr['file_date'].nunique()

114

In [29]:
df_uid_idl_mapping_2020_impr.head(2)

,Customer_Link,User ID,file_date
0,XY1468o1xIf7Jby50QJZZJVwoJ27BzXAi8uNgHxdHMJfL9RVQ,AMsySZa6UdLeYc7XcC9M7fcEXliW,20200101
1,XY1468VcODxZFpl4kAxJhAtPWHz38JU1XM_ff_o74EY9TzIns,AMsySZZaPfN25_zrxWNGzQprbByK,20200101


In [30]:
df_uid_idl_mapping_2020_impr['Customer_Link'].nunique()

5906890

In [31]:
df_uid_idl_mapping_2020_impr['User ID'].nunique()

6722839